# Training Predictive Model

### Import libraries

In [ ]:
import datetime, json, random, IPython, pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.image as mpimg
import torch, pytorch_lightning as pl
from ray import tune
from pytorch_lightning.loggers import TensorBoardLogger

from models import GazeDataModule, SingleModel, EyesModel, FullModel
from utils  import (
    get_config,
    tune_asha,
    get_best_results,
    save_model,
    plot_asha_param_grid,
    plot_parallel_param_loss,
    latest_tune_dir,
    _build_datamodule,
    _build_model,
    predict_screen_errors,
)

# project settings
SETTINGS, COLOURS, EYETRACKER, TF = get_config("config.ini")

%load_ext autoreload
%autoreload 2


To fix the "Import 'mpimg' could not be resolved" error, you need to install the `matplotlib` library, as `mpimg` is part of `matplotlib.image`. Use the `%pip install` magic command in Jupyter Notebook to install the required package.



Made changes.

To fix the "Import 'mpimg' could not be resolved" error, you need to install the `matplotlib` library, as `mpimg` is part of `matplotlib.image`. Use the `%pip install` magic command in Jupyter Notebook to install the required package.



Made changes.

### Dataset information

In [ ]:
df = pd.read_csv("data/positions.csv")
region_map = np.load("data/region_map.npy").T

print(f"# of samples: {len(df)}")
coverage = np.count_nonzero(region_map > 0) / region_map.size * 100
print(f"Coverage: {coverage:.2f}% of screen surface")
print(f"Crop size: {SETTINGS['image_size']} x {SETTINGS['image_size']} px")

### Fine tuning

In [ ]:
search_space = {
    "seed":  tune.randint(0, 10000),
    "bs":    tune.choice([64, 128, 256]),
    "lr":    tune.loguniform(1e-4, 3e-3),
    "face_channels"     : tune.choice([(32, 64, 128), (48, 96, 192)]),
    "eye_channels"      : tune.choice([(32, 64, 128), (48, 96, 192)]),
    "head_pos_channels" : tune.choice([(16, 32, 64),  (24, 48, 96)]),
    "hidden": tune.choice([256, 512, 768]),
}

analysis = tune_asha(
    search_space   = search_space,
    train_func     = "full",
    name           = "full/tune",
    img_types    = ["face_aligned", "l_eye", "r_eye", "head_pos", "head_angle"],
    num_samples    = 36,
    num_epochs     = 15,
    data_dir     = Path.cwd() / "data",
    seed           = 87,
)

In [ ]:
plot_asha_param_grid(
    analysis,
    params=("bs", "lr",
            "face_channels", "eye_channels", "head_pos_channels",
            "hidden"),
    save_path="media/images/final_full_explore_scatter.png",
)

In [ ]:
plot_parallel_param_loss(
    analysis,
    cols=("bs", "lr",
          "face_channels", "eye_channels", "head_pos_channels",
          "hidden"),
    save_path="media/images/final_full_explore_parallel.png",
)

### Training

In [ ]:
start_time = datetime.datetime.now().strftime("%Y-%b-%d %H-%M-%S")

tune_dir = Path.cwd() / "logs" / "full"
best_cfg = get_best_results(latest_tune_dir(tune_dir))
pl.seed_everything(best_cfg["seed"])

dm = GazeDataModule(
    data_dir = Path.cwd() / "data",
    batch_size = best_cfg["bs"],
    img_types = ["face_aligned", "l_eye", "r_eye", "head_pos", "head_angle"],
    seed = best_cfg["seed"],
)

model = _build_model(best_cfg, [
    "face_aligned", "l_eye", "r_eye", "head_pos", "head_angle"
])

trainer = pl.Trainer(
    max_epochs = 100,
    accelerator = "auto",
    devices = "auto",
    precision = "bf16-mixed",
    logger = TensorBoardLogger(
        save_dir = Path.cwd() / "logs",
        name     = f"full/final/{start_time}",
        log_graph = True,
    ),
    callbacks = [
        pl.callbacks.ModelCheckpoint(
            filename = "best",
            monitor  = "val_loss",
            mode     = "min",
            save_last = True,
            save_top_k = 1,
        )
    ],
)

trainer.fit(model, datamodule=dm)
best_path = trainer.checkpoint_callback.best_model_path
state = torch.load(best_path, map_location="cpu", weights_only=False)
model.load_state_dict(state["state_dict"])

out_dir = Path.cwd() / "logs" / "full" / "final" / start_time
out_dir.mkdir(parents=True, exist_ok=True)

save_model(
    model.cpu(),
    best_cfg,
    out_dir / "eyetracking_model.pt",
    out_dir / "eyetracking_config.json",
)

### Model Evaluation

In [ ]:
test_results = trainer.test(ckpt_path="best", datamodule=dm)[0]

loss = test_results["test_loss_epoch"]
mae  = test_results["test_mae_epoch"]

mse  = test_results.get("test_mse_epoch",  test_results.get("test_mse"))
rmse = test_results.get("test_rmse_epoch", test_results.get("test_rmse"))

print("────────  Test set  ────────")
print(f"MSE   : {mse:8.2f}  px²")
print(f"RMSE  : {rmse:8.2f}  px")
print(f"MAE   : {mae:8.2f}  px")
print(f"Loss  : {loss:8.2f}  (Smooth-L1)")

In [ ]:
predict_screen_errors(
    "face_aligned", "l_eye", "r_eye", "head_pos", "head_angle",
    path_model  = out_dir/"eyetracking_model.pt",
    path_config = out_dir/"eyetracking_config.json",
    path_plot   = out_dir/"error_heatmap_full.png",
    path_errors = out_dir/"errors.npy",
    steps       = 10,
)